## RAG 성능 올리는 기술(LLM 기반의 리랭킹)

###1. LLM기반의 리랭킹
LLM 기반 리랭킹은 대규모 언어 모델 (LLM) 을 사용해 초기 검색 결과를 다시 평가하고, 사용자의 질문과
가장 관련이 높은 순서로 결과를 정렬하는 방법입니다. 초기 검색에서는 임베딩 모델 (주로 BERT 류의 문
서 임베딩 모델) 을 이용한 유사도 계산이 사용되지만 LLM 과 비교하면 상대적으로 정확도가 떨어집니다.
따라서 LLM 기반 리랭킹은 임베딩 모델의 초기 검색 결과를 보완하기 위해 사용합니다. 이 과정은 다음과
같이 진행됩니다

- 초기 검색 결과 준비: 임베딩 기반의 검색 시스템에서 반환된 상위 몇 개의 결과 (예: Top‑4, Top‑6 등) 를 입력으로 사용합니다.

- LLM 을 통한 검색 결과 순위 조정: LLM 은 초기 검색 결과를 전체적으로 검토합니다. 예를 들어, 사용자
가 “초보자를 위한 요리법” 을 검색했을 때, 임베딩 모델을 통한 검색 결과로 “고급 프랑스 요리 과정” 과
“간단한 샌드위치 만들기” 가 반환되었다면, 초기 임베딩 모델을 통한 검색에서는 “고급 프랑스 요리 과
정” 이 더 높은 순위를 받았을 수 있습니다. “요리 과정” 과 “요리법” 이 비슷한 의미를 가지고 있기 때문입니다. 하지만 LLM 은 사용자의 질문과 내용을 의미적으로 비교한 후, “간단한 샌드위치 만들기” 가 “초보자” 라는 점을 감안하면, 더 적합하다고 판단하고 더 높은 점수를 줄 수 있습니다.

-  점수 기반 재정렬: LLM 이 새로 계산한 점수를 기준으로 기존의 임베딩 모델을 통한 검색 결과를 다시 정렬합니다. 예를 들어, LLM 이 관련성 점수를 1~10 으로 매겼다면, 점수가 높은 순서대로 검색 결과를 배열
합니다. 이때 점수가 낮은 결과는 최종 목록에서 제외될 수도 있습니다. 실습을 통해 자세히 알아보겠습
니다.

각 문서마다 이러한 점수 계산이 필요하다 보니, LLM 호출 횟수가 증가하게 될 수 있음을 유의해야 합니
다. 이를 구체적으로 살펴보기 위해 임베딩 검색으로 상위 4 개의 문서를 가져온 후 RAG 를 수행하는 과정
을 비교해보겠습니다

‑ 벡터 검색으로 문서 4 개를 찾아옵니다

‑ LLM 호출 1 회: 검색된 4 개 문서 전체를 바탕으로 답변을 생성합니다

‑ 총 LLM 호출: 1 회

‑ 벡터 검색으로 문서 4 개를 찾아옵니다

‑ LLM 호출 4 회: 각 문서와 질문의 관련성을 1‑10 점 사이로 평가합니다

‑ LLM 호출 1 회: 관련성 점수가 높은 상위 2개 문서만으로 답변을 생성합니다

‑ 총 LLM 호출: 5 회

LLM 기반 리랭킹은 정확도는 높아지지만 각 문서의 관련성 평가와 최종 답변 생성까지 총 5 번의 LLM 호
출이 필요해 비용과 시간이 증가하게 됩니다. 이제 본격적으로 실습을 진행해보겠습니다.


###2. 패키지 설치 및 OpenAI키 값 설정
랭체인 패키지와 벡터 데이터베이스를 위한 langchain_chroma, PDF 를 읽기 위한 pypdf 를
설치합니다.

In [ ]:
!pip install langchain_openai langchain_community langchain_chroma pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 86.7 MB/s eta 0:00:

실습을 위해 필요한 패키지들을 임포트합니다.

In [ ]:
import os
import urllib.request
import json
from typing import List
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.schema import Document
import requests

OpenAI의 Key값을 셋팅합니다.

In [ ]:
os.environ['OPENAI_API_KEY'] = 'Openai_api_key'

###2. 데이터 다운로드
사용할 데이터는 2023_북한인권보고서.pdf입니다.

In [ ]:
# 분석할 PDF 파일을 웹에서 다운로드.
url = "https://github.com/llama-index-tutorial/llama-index-tutorial/raw/main/ch07/2023_%EB%B6%81%ED%95%9C%EC%9D%B8%EA%B6%8C%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf"
filename = "2023_북한인권보고서"

response = requests.get(url)
with open(filename, "wb") as f:
  f.write(response.content)

print(f"{filename} 다운로드 완료")

2023_북한인권보고서 다운로드 완료


###3. 거대언어모델과 임베딩 설정
랭체인을 사용하여 사용할 각종 설정들의 값을 정합니다.

In [ ]:
# LangChain의 LLM과 임베딩 모델 설정
llm = ChatOpenAI(model = 'gpt-4o', temperature=0.2) # gpt-4o를 언어 모델로 사용
embed_model = OpenAIEmbeddings(model = 'text-embedding-3-large') # 임베딩 모델 사용

# 문서 분할 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300, # 문서를 300자 단위로 분할
    chunk_overlap = 100, # 문맥 유지를 위해서 청크간 100자 중복
)

#PDF 문서를 읽고 벡터 인덱스 생성
loader = PyPDFLoader("2023_북한인권보고서") # PDF 문서 로더
documents = loader.load() # 문서에서 텍스트 추출
chunks = text_splitter.split_documents(documents) # 문서 분할
vector_store = Chroma.from_documents(chunks, embed_model) # - 각 문서 청크를 embed_model로 벡터화 - 벡터와 원본 문서를 Chroma에 저장


랭체인을 통해 각 구성 요소를 개별적으로 설정합니다. ChatOpenAI를 통해 이번에 사용할 LLM을 OpenAI의 GPT-4o로 설정하고, OpenAIEmbeddings를 통해 앞으로 사용할 임베딩 모델을 OpenAI의 text-embedding-3-large로 설정하였습니다.

문서처리를위한chunk_size=300과chunk_overlap=100설정은RecursiveCharacterTextSplitter에서 적용됩니다. 길이 300 기준으로 문서를 나누고 길이 100 자의 문자열을 중복되게 하여, 나중에 문서를 검색할 때 문맥이 끊기지 않도록 합니다

이렇게 설정한 후,PyPDFLoader로PDF파일을읽어들이고load()로 텍스트를 추출합니다. 추출된 텍스트는 text_splitter.split_documents()를 통해 분할되고, Chroma.from_documents()를 통해 벡터 인덱스로 변환되는데, 이 과정에서 위에서 설정한 임베딩 모델과 청크 설정이 모두 적용됩니다. 이렇게 생성된 인덱스는 이후 SemanticRanker 클래스에서 벡터 유사도 검색의 기반이 됩니다.

###4. 리랭킹 구현하기
이제 리랭킹을 위한 모듈 두 가지를 구현합니다. 먼저 구현할 DocumentScorer는 문서 검색 결과를 LLM 으로 재평가하는 리랭킹을 수행하는 모듈입니다.

In [ ]:
class DocumentScorer:
  # LLM을 사용해 문서의 관련성을 정밀하게 평가하고 점수를 매기는 클래스

  def __init__(self, llm):
    self.llm = llm

  def evaluate_document(self, query: str, content: str) -> float:
    # LLM을 사용해 문서와 쿼리 간의 의미적 관련성을 1-10점으로 평가
    prompt = f"""
    아래 주어진 질문과 문서의 관련성을 평가해 주세요.

    [평가 기준]
    - 문서가 질문에서 요구하는 정보를 직접적으로 포함하면 8-10점
    - 문서가 질문과 관련된 맥락을 포함하지만 직접적인 답이 아니면 4-7점
    - 문서가 질문과 거의 관련이 없으면 1-3점

    [주의 사항]
    - 단순히 비슷한 단어가 등장하는 것은 높은 점수의 근거가 될 수 없습니다.
    - 질문의 의도와 문맥을 정확히 파악하여 평가해주세요
    - 시간, 장소, 수치 등 구체적인 정보의 일치 여부를 중요하게 고려해 주세요

    질문: {query}
    문서: {content}

    응답은 반드시 다음 JSON 형식이어야 합니다. 백틱은 쓰지 마십시오.:
    {{"relevance_score": float}}
    """

    try:
      # LLM에 프롬프트를 전송하고 JSON 형식의 응답을 받음
      response = self.llm.invoke(prompt)
      # 응답에서 relevance_score 값을 추출
      score = json.loads(response.content)["relevance_score"]
      # 점수를 float로 변환하여 반환
      return float(score)

    except Exception as e:
      print(f"Error occurred: {str(e)}")
      return 5.0 # 에러 발생시 중간 점수로 처리하여 시스템 안정성 유지

  def postprocess_documents(self, documents: List[Document], query:str) ->List[Document]:
    # 벡터 검색으로 찾은 4개 문서를 LLM으로 재평가 하여 최적의 2개를 선택
    print('\n====LLM이 4개의 검색 결과에 대해서 관련성을 평가합니다.====')
    scored_docs = []
    for doc in documents:
      # 현재 처리 중인 문서에서 순수 텍스트 컨텐츠만 추출
      content = doc.page_content
      # LLM으로 문서 관련성 점수 계산 (1-10 사이 점수)
      score = self.evaluate_document(query, content)
      # 디버깅/ 모니터링을 위해 각 문서의 내용과 점수를 출력
      print(f"\nLLM 기반의 평가: \n{content}\n =>점수: {score}\n")
      # 현재 문서와 계산된 점수를 튜플로 저장
      scored_docs.append((doc, score))

    # 모든 문서를 점수 기준 내림차순으로 정렬하고 상위 2개만 선택하여 반환
    ranked_docs = sorted(scored_docs, key=lambda x: x[1], reverse = True)
    return [doc for doc, _ in ranked_docs[:2]]

먼저 DocumentScorer 내부의 evaluate_document를 주목해봅시다. 내부에 LLM 이 검색 결과에
대해서 점수를 측정하는 프롬프트가 작성되어져 있습니다. prompt라는 변수에 작성된 이 프롬프트에
는 입력받은 질문과 검색된 문서의 관련성을 1 점부터 10 점까지의 점수로 평가합니다. 프롬프트에서는
LLM 에게 문서가 질문이 원하는 정보를 직접 포함하면 8‑10 점, 관련 맥락만 있으면 4‑7 점, 거의 관련 없
으면 1‑3 점을 주도록 지시하고 있습니다.
실제로 DocumentScorer의 실행 순서는 내부의 postprocess_documents가 먼저 실행되고,
postprocess_documents 내부에서 또 다른 메소드인 evaluate_document를 호출하는 구조입
니다. postprocess_documents에서는 사용자의 질문과 해당 질문으로부터 임베딩 모델이 찾은 초
기 검색 결과로 4 개의 문서가 전달됩니다. 이 4 개의 검색 결과가 위에서 설명한 evaluate_document
로 전달하고, evaluate_document가 사용자 질문과 각각의 문서에 대해서의 연관성에 대한 평가를
마치면, LLM 으로부터 받은 점수와 평가 과정을 출력합니다. 그 후 높은 점수 기준으로 정렬하여 가장 높은 점수를 받은 2 개의 문서만 반환합니다. 이렇게 초기 검색 결과 4 개는 LLM 으로 다시 관련성 점수가 측
정되어 관련성 점수가 높은 2 개의 문서만 반환되는 리랭킹 과정을 거칩니다.

지금까지 검색 결과 4 개에 대해서 리랭킹 과정을 수행하는 리랭킹 모듈 DocumentScorer에 대해서
설명했습니다. 이번에는 검색 결과 4 개를 입력으로 위의 리랭킹 모듈을 실제로 호출하는 역할을 하는
SemanticRanker에 대해서 알아봅시다.

In [ ]:
class SemanticRanker:
  # 벡터 검색 결과에 LLM기반 의미적 평가를 적용하여 최적의 문서를 선별하는 시스템
  def __init__(self, vector_store, scorer):
    # 생성자에서 벡터 검색용 저장소와 LLM 기반 문서 평가가 인스턴스를 받아 저장
    self.vector_store = vector_store # 벡터 검색용 저장소
    self.scorer = scorer

  def retrieve(self, query: str) -> List[Document]:
    # 벡터 검색으로 유사도 기반 후보 문서 4개를 추출하고 LLM으로 재평가
    vector_results = self.vector_store.similarity_search(query, k=4)

    # 초기 벡터 검색 결과를 디버깅/ 분석용으로 출력
    print("\n==== 실제 검색 결과 (Top4) ====")
    for i, doc in enumerate(vector_results, 1):
      print(f"\n검색 문서 {i}:")
      print(doc.page_content)

    # LLM으로 문서들을 재평가하고 재정렬하여 최적의 2개 선택
    reranked_results = self.scorer.postprocess_documents(vector_results, query)

    # 최종 선별되 문서를 디버깅 /분석용으로 출력
    print("\n==== LLM의 리랭킹 결과(Top2) ====")
    for i, doc in enumerate(reranked_results, 1):
      print(f"\n검색 문서 {i}:")
      print(doc.page_content)

    return reranked_results

실제실행순서는SemanticRanker의retrieve에서DocumentScorer의postprocess_documents
를 호출하는 순서입니다. SemanticRanker는 벡터 검색 후에 LLM 기반 평가를 실행하는 모듈입니다.
실제 검색이 어떻게 수행되는지 설명해봅시다.

SemanticRanker가 실 행 되 면 retrieve 메 소 드 가 호 출 되 며 내 부 에 서 vector_store.
similarity_search를 통해 먼저 벡터 기반 검색을 수행하여 상위 4 개의 문서를 가져옵니다. 이때
k=4 파라미터로 상위 4 개만 추출하도록 지정했습니다. 추출된 4 개의 문서는 디버깅을 위해 “실제 검색
결과 (Top 4)” 라는 제목으로 출력됩니다.

그 다음, 앞서 설명한 DocumentScorer의 postprocess_documents를 호출하여 이 4 개의 문서
를 LLM 으로 재평가합니다. 재평가가 완료되어 상위 2 개의 문서가 선택되면 “LLM 의 리랭킹 결과 (Top
2)” 라는 제목으로 최종 선택된 문서들을 출력하고 이를 반환합니다.

이런 방식으로 SemanticRanker는 벡터 검색으로 1 차 필터링을 하고, LLM 으로 2 차 정밀 평가를 수
행하여 사용자 질문에 가장 적합한 문서를 찾아내는 2 단계 검색을 구현하고 있습니다.

이제 위에서 설명한 두 개의 리랭킹 모듈 DocumentScorer와 SemanticRanker를 실제 객체로 선
언합니다.

In [ ]:
# 문서 평가 및 검색 시스템 선언 (초기화)
scorer = DocumentScorer(llm) # LLM기반 문서 평가기 생성
ranker = SemanticRanker(vector_store, scorer) # 벡터 검색어와 LLM평가를 결합한 시스템 생성

모듈 선언 단계에서는 두 단계의 객체 생성이 일어납니다. DocumentScorer 객체를 scorer라
는 이름으로 생성하고, 이 scorer를 실제로 실행할 SemanticRanker에 전달하여 벡터 검색과 리
랭킹을 결합된 시스템 객체 ranker를 생성합니다. 리랭킹 모듈들을 실제로 실행해보겠습니다.
generate_final_answer 함수를 정의하여 선택된 문서들로 최종 답변을 생성하는 기능을 구현합
니다.


In [ ]:
# 최종 답변 생성 함수
def generate_final_answer(query: str, documents: List[Document]) -> str:
  context ="\n\n".join([doc.page_content for doc in documents])

  prompt = f""" 다음 검색 결과를 바탕으로 질문에 답변해주세요.
  검색 결과의 정보를 최대한 사용하고, 없는 정보는 답변하지 마세요.

  검색 결과:
  {context}

  질문: {query}

  답변:"""

  response = llm.invoke(prompt)
  return response.content

# 실제 쿼리 실행
query = "19년 말 평양시 소재 기업소에서 달마다 배급받은 음식"
print(f"\n질문: {query}")

# 리래킹을 통해 최적의 문서 2개 선택
best_documents = ranker.retrieve(query)

# 선택된 문서로 최종 답변 생성
final_answer = generate_final_answer(query, best_documents)
print(f"\n최종 답: {final_answer}")


질문: 19년 말 평양시 소재 기업소에서 달마다 배급받은 음식

==== 실제 검색 결과 (Top4) ====

검색 문서 1:
화 또는 쌀이나 기름 등 현물로 지급하였다고 한다. 2019년 평양
의 외화벌이 사업소에서는 보수 50달러를 월 2회로 나누어 현금으
로 지급하였다고 하는 사례가 있었고, 평양 외화벌이 식당에서는 매

검색 문서 2:
파악되었다. 따라서 기관·기업소의 상황에 따라 식량배급량, 주기, 
곡식종류에 상당한 차이가 있는 것으로 나타났다. 외화벌이 기관 등
에는 식량배급이 원활하게 이뤄지고 있었다는 증언이 수집되었다. 
2019년 평양시에서 기업소 운전원으로 일하였던 노동자는 매월 쌀·
설탕·기름·야채·돼지고기 등을 배급받아 식량이 부족하지 않았다는 
증언과 2019년 중앙당 산하의 기업소에서 매월 쌀 6㎏ 정도, 기름 5
ℓ, 설탕 2㎏, 맛내기 2봉지, 돼지고기 2㎏, 닭고기 1마리 정도 받았

검색 문서 3:
가배급을 선택하고, 잘사는 기업소들은 기업소 자체 배급을 선택합
니 다. 세대주가 직장에 다닐 경우 세대주만 직장에서 배급을 받고 
가족들은 국가배급소에서 배급을 받습니다. 평양시와 자강도는 대
체로 다 줬는데 다른 지역은 배급이 잘 안되고 배급제가 없어졌다는 
소리를 들었습니다. ”
국가배급의 주기, 양, 곡물의 종류 등에서 평양시와 지방의 차이
가 크게 나고 있었다. 식량배급이 비교적 원활하게 작동하는 지역은 
평양시로 보이는데, 2017년 어머니가 지역배급 대상자로 배급표가

검색 문서 4:
한 달을 생활하기에 부족한 금액이었다고 하였다. 2018년 양강도의 
무역사업소에서는 1년치 노동 보수와 배급을 한 번에 지급하였다고 
하는데, 지급된 금액은 노동자 1명에게 1,800위안으로 약 300만원 
정도였다고 하였다. 2019년 양강도의 합영회사는 노동자에게 매달 
9~12만원의 보수를 지급하고, 1년에 한번 쌀 25kg을 지급하였다는 
진술이 있었다. 또한 2020년 합영회사에서는 보수를 성과만큼 받았
다고 하는데, 숙련공

generate_final_answer 함수는 검색된 문서들의 내용을 하나의 컨텍스트로 합치고, LLM 에게 이
정보만을 사용해서 답변하도록 지시하는 프롬프트를 만듭니다.

이후ranker.retrieve() 에실제사용자의질문을입력하면벡터검색, 리랭킹 과정이 순차적으로 이루어져 최적의 2 개 문서가 반환됩니다. 예시로 “19 년 말 평양시 소재 기업소에서 달마다 배급받은 음식”이라는 질문을 ranker.retrieve()로 전달합니다. 이때 내부적으로는 먼저 SemanticRanker가
벡터 검색으로 4 개 문서를 찾고 이후 DocumentScorer가 LLM 으로 재평가하여 2 개를 선택한 후,generate_final_answer가 LLM 이 관련성이 가장 높다고 판단한 최종 2 개의 문서들을 바탕으로
응답을 생성하여 반환합니다.

먼저 “19 년 말 평양시 소재 기업소에서 달마다 배급받은 음식” 이라는 질문에 대해 벡터 검색이 실행되어 4 개의 문서를 찾아왔습니다. LLM 은 이 4 개 문서를 평가했습니다.
-  외화벌이 사업소 보수 관련 문서: 3.0 점
-  2019 년 평양시 기업소 구체적 배급량 문서: 9.0 점
-  평양시 배급 시스템 일반 설명 문서: 6.0 점
-  양강도 합영회사 보수 관련 문서: 1.0 점

LLM 이 가장 높은 점수 (9.0) 를 준 문서는 2019 년 평양시 기업소의 구체적인 배급량을 언급한 문서였습
니다. 이는 질문이 요구한 시기 (19 년), 장소 (평양시), 대상 (기업소), 내용 (배급받은 음식) 을 모두 정확히 포함하고 있기 때문입니다. 두 번째로 높은 점수 (6.0) 를 받은 문서는 평양시의 배급 시스템에 대한 일반적인 설명이었습니다. 이 문서는 구체적인 배급량을 제시하지는 않지만 평양시 배급 현황에 대한 맥락 정보를 제공하므로 중간 정도의 관련성을 인정받았습니다.

반면 양강도 합영회사에 대한 문서는 1.0 점으로 가장 낮은 점수를 받았습니다. 이 문서는 2019 년 정보를
포함하고 있지만 평양시가 아닌 양강도에 관한 내용이고, 기업소가 아닌 합영회사에 관한 내용이며, 구체적인 음식 배급보다는 현금 보수와 연간 쌀 지급에 초점을 맞추고 있어 질문과의 관련성이 낮다고 평가되
었습니다.

최종적으로 점수 순으로 상위 2 개 문서가 선택되어 응답이 생성되었고, 가장 관련성 높은 문서의 구체적
인 배급량 정보 (쌀 6kg, 기름 5ℓ, 설탕 2kg, 맛내기 2 봉지, 돼지고기 2kg, 닭고기 1 마리) 를 답변으로 제시
했습니다.

전체적으로 LLM 리랭킹이 질문의 의도에 적절하게 작동했음을 확인할 수 있습니다.
